Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 3 <br>
Date: 11/20/2021 <br>

You are to apply the following techniques to mine this data set:
- Association Rules
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

Also, you are to use Principle Component Analysis to reduce the number of attributes
(aka features)

Please answer the following questions:

1. What can you deduce from the data set? (in otherwords, what attribute values are indicative of "success")
2. Which mining techniques yielded the best results for what? How do you define "best results"? Please
include tables and/or graphs to justify your statement

Note: While numeric data is desired for classification/clustering, you need strings that look like market basket
data (sets of item names) for Association Rules. Numeric data can be grouped into names like
"lowValue, midValue, midValue, mediumHighValue, highValue" etc. See the following code for possible
way of doing this (not necessarily the best):

What to turn in:
- a pdf file with the answers to these three questions
- Your code used to answer these questions


In [2]:
import pandas as pd


In [5]:
df =  pd.read_csv("bank.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
